In [73]:
def parse_conf_from_str(s):    
    import re
    rs = {}
    for m in [m for m in re.finditer('\[([^]]+)\]([^\[]+)', re.sub('#[^\[].*','', s) )]:
        items = [re.sub('\s*#.*','', l).strip() for l in m.group(2).strip().splitlines()]
        rs[m.group(1)] = [i for i in items if i]
    return rs

s='''
[servers]
# comments here
us # 1st
dummy # test except
hk # 3rd

#[cmd_root]  # "#[" is not comment
echo "linux shell commands"
'''
conf = parse_conf_from_str(s)
print(conf)
g = conf['servers']
g

{'servers': ['us', 'dummy', 'hk'], 'cmd_root': ['echo "linux shell commands"']}


['us', 'dummy', 'hk']

In [72]:
def parse_conf_from_file(conf_file):
    try:
        with open(conf_file,'r') as f:
            s = f.read()
        return parse_conf_from_str(s)
    except:
        print("Exception: ", sys.exc_info()[0])
        return None
     
import sys
conf_file = "/data/batch_ssh_servers.conf"
conf = parse_conf_from_file(conf_file)
print(conf)

Exception:  <class 'FileNotFoundError'>
None


In [64]:
connect_kwargs={
    "key_filename": "/data/id_rsa",
}
    
from fabric import Connection

import fabric
fabric.ThreadingGroup

conn_list = [ Connection(h, port=10220,  connect_kwargs=connect_kwargs) for h in g ]

from fabric import ThreadingGroup as Group
pool= Group.from_connections(conn_list)
pool

[<Connection host=us port=10220>,
 <Connection host=notworking port=10220>,
 <Connection host=hk port=10220>]

In [12]:
from fabric.exceptions import GroupException

def append_GroupResult(GroupResult_list, pool, cmd_list, **kwargs):
    if len(pool) == 0: return GroupResult_list
    
    if type(cmd_list) == str:
        cmd_list = cmd_list.strip().splitlines()
        
    for cmd in cmd_list:
        try:
            GroupResult_list.append(pool.run(cmd, **kwargs))
        except GroupException as e:
            GroupResult_list.append(e.result)
            
    return GroupResult_list


def group_run(pool, cmd_list, **kwargs):    
    GroupResult_list = append_GroupResult([], pool, 'hostname')
    #pool_work = Group.from_connections([c for c in pool if type(GroupResult_list[0].get(c)) == fabric.runners.Result])
    pool_work = Group.from_connections([c for c in pool if c.is_connected])    
    return append_GroupResult(GroupResult_list, pool_work, cmd_list, **kwargs)

cmds="""
date
ls /etc/hosts*
sleep 3
date
"""
run_rs = group_run(pool,cmds,hide='out')
run_rs

HK1C1GTX
bwg2c2g


[{<Connection host=hk port=10220>: <Result cmd='hostname' exited=0>,
  <Connection host=notworking port=10220>: socket.gaierror(-2,
                  'Name or service not known'),
  <Connection host=us port=10220>: <Result cmd='hostname' exited=0>},
 {<Connection host=hk port=10220>: <Result cmd='date' exited=0>,
  <Connection host=us port=10220>: <Result cmd='date' exited=0>},
 {<Connection host=hk port=10220>: <Result cmd='ls /etc/hosts*' exited=0>,
  <Connection host=us port=10220>: <Result cmd='ls /etc/hosts*' exited=0>},
 {<Connection host=hk port=10220>: <Result cmd='sleep 3' exited=0>,
  <Connection host=us port=10220>: <Result cmd='sleep 3' exited=0>},
 {<Connection host=hk port=10220>: <Result cmd='date' exited=0>,
  <Connection host=us port=10220>: <Result cmd='date' exited=0>}]

In [13]:
def run_rs_to_dict_list(run_rs, splitlines=True):
    run_rs_list = []
    for rs in run_rs:
        for k,v in rs.items():
            item = {'Host': k.host}
            if type(v) == fabric.runners.Result:
                d = vars(v)
                for k in ['command','exited']: 
                    item[k] = d[k]
                for k in ['stdout','stderr']: 
                    if splitlines: item[k] = d[k].strip().splitlines()
                    else:          item[k] = d[k].strip()
            else:
                item['ConnError'] = str(v)

            run_rs_list.append(item)

    return run_rs_list
        
import json

with open("fabric2_result.json", "w") as f:
    json.dump(run_rs_to_dict_list(run_rs), f, indent=4)        

In [14]:
!cat fabric2_result.json

[
    {
        "Host": "hk",
        "command": "hostname",
        "exited": 0,
        "stdout": [
            "HK1C1GTX"
        ],
        "stderr": []
    },
    {
        "Host": "us",
        "command": "hostname",
        "exited": 0,
        "stdout": [
            "bwg2c2g"
        ],
        "stderr": []
    },
    {
        "Host": "notworking",
        "ConnError": "[Errno -2] Name or service not known"
    },
    {
        "Host": "hk",
        "command": "date",
        "exited": 0,
        "stdout": [
            "Wed May  1 23:27:16 CST 2019"
        ],
        "stderr": []
    },
    {
        "Host": "us",
        "command": "date",
        "exited": 0,
        "stdout": [
            "Wed May  1 11:27:16 EDT 2019"
        ],
        "stderr": []
    },
    {
        "Host": "hk",
        "command": "ls /etc/hosts*",
        "exited": 0,
        "stdout": [
            "/etc/hosts",
            "/etc/hosts.allow",
            "/etc/hosts.deny"
        ],
        "st

In [15]:
import pandas as pd
df = pd.DataFrame(run_rs_to_dict_list(run_rs, splitlines=False)).fillna('')
df = df.set_index('Host', append=True).sort_index(level=1).reset_index(level=1)
df

,Host,ConnError,command,exited,stderr,stdout
0,hk,,hostname,0,,HK1C1GTX
3,hk,,date,0,,Wed May 1 23:27:16 CST 2019
5,hk,,ls /etc/hosts*,0,,/etc/hosts\n/etc/hosts.allow\n/etc/hosts.deny
7,hk,,sleep 3,0,,
9,hk,,date,0,,Wed May 1 23:27:21 CST 2019
2,notworking,[Errno -2] Name or service not known,,,,
1,us,,hostname,0,,bwg2c2g
4,us,,date,0,,Wed May 1 11:27:16 EDT 2019
6,us,,ls /etc/hosts*,0,,/etc/hosts\n/etc/hosts.allow\n/etc/hosts.deny
8,us,,sleep 3,0,,


In [17]:
df[df['command'] == 'date']

,Host,ConnError,command,exited,stderr,stdout
3,hk,,date,0,,Wed May 1 23:27:16 CST 2019
9,hk,,date,0,,Wed May 1 23:27:21 CST 2019
4,us,,date,0,,Wed May 1 11:27:16 EDT 2019
10,us,,date,0,,Wed May 1 11:27:21 EDT 2019


In [18]:
df[df['command'].str.contains('ls', regex=False)]

,Host,ConnError,command,exited,stderr,stdout
5,hk,,ls /etc/hosts*,0,,/etc/hosts\n/etc/hosts.allow\n/etc/hosts.deny
6,us,,ls /etc/hosts*,0,,/etc/hosts\n/etc/hosts.allow\n/etc/hosts.deny


In [19]:
for c in pool:
    print(c.is_connected)

True
False
True


In [334]:
pool.close()